In [1]:
import numpy as np
import xarray as xr
import glob

from dask.distributed import Client

client = Client(n_workers=14)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 14
Total threads: 14,Total memory: 63.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44567,Workers: 14
Dashboard: /proxy/8787/status,Total threads: 14
Started: Just now,Total memory: 63.00 GiB
Comm: tcp://127.0.0.1:36997,Total threads: 1
Dashboard: /proxy/39219/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:38921,


In [2]:
base = '/g/data/ik11/outputs/access-om2-025/025deg_era5_iaf'

T = xr.open_mfdataset(sorted(glob.glob(base + '/output***/ocean/ocean-3d-temp-1-monthly*.nc')), concat_dim="time", combine="nested",
                             data_vars='minimal', coords='minimal', compat='override',
                             parallel=True,chunks={'time':1}).temp

area = xr.open_dataset(glob.glob(base + '/output***/ocean/ocean-2d-area_t.nc')[0]).area_t

mask = ~np.isnan(T.isel(time=0)).load()
total_area = area.where(mask).sum(['xt_ocean','yt_ocean']).load()
Tz = (T*area).where(mask).sum(['xt_ocean','yt_ocean'])/total_area

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.04/lib/python3.9/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
module 'ecmwflibs' has no attribute 'find'
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.04/lib/python3.9/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
module 'ecmwflibs' has no attribute 'find'
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.04/lib/python3.9/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
module 'ecmwflibs' has no attribute 'find'
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.04/lib/python3.9/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading faile

In [3]:
Tz

<xarray.DataArray (time: 528, st_ocean: 50)>
dask.array<truediv, shape=(528, 50), dtype=float32, chunksize=(1, 50), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 1.152 3.649 6.565 ... 5.034e+03 5.254e+03
  * time      (time) datetime64[ns] 1980-01-16T12:00:00 ... 2023-12-16T12:00:00

In [ ]:
# Do 3D computations:
Tresults = []
for yi in range(int(len(Tz.time)/12)):

    res = Tz.isel(time=slice(yi*12,(yi+1)*12))
    res.load()
    Tresults.append(res)

OM2_TSz = xr.concat(Tresults,dim='time').rename('Temperature').to_dataset()